In [1]:
!pip install langchain langchain_community openai sentence_transformers faiss-cpu langchain_openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 695.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.7/390.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 923.4/923.4 kB 17.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 28.5

In [2]:
# import
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_openai import OpenAIEmbeddings
import openai
import pandas as pd
import csv
import re
import faiss
import numpy as np

/Users/yuliiamaksymyuk/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
loader = CSVLoader(file_path="./rag-data/hutsul-base.csv", source_column="hutsul_sentence")

data = loader.load()

In [4]:
data[:10]

[Document(page_content='hutsul_sentence: У дєді з рєду дітий я двайціть и чєтвертий.', metadata={'source': 'У дєді з рєду дітий я двайціть и чєтвертий.', 'row': 0}),
 Document(page_content='hutsul_sentence: А мижи живими дітми одинайцітий и найменчий, але ни випорток, бо післідь мене було ишє шестеро діточьок, лиш уни маленькими вивмерали.', metadata={'source': 'А мижи живими дітми одинайцітий и найменчий, але ни випорток, бо післідь мене було ишє шестеро діточьок, лиш уни маленькими вивмерали.', 'row': 1}),
 Document(page_content='hutsul_sentence: Ріс я пестуном у хаті, прото найменчя дитина у старині.', metadata={'source': 'Ріс я пестуном у хаті, прото найменчя дитина у старині.', 'row': 2}),
 Document(page_content='hutsul_sentence: Розвезений, шо най Бог боронит.', metadata={'source': 'Розвезений, шо най Бог боронит.', 'row': 3}),
 Document(page_content='hutsul_sentence: Вара було шо ми ирчі.', metadata={'source': 'Вара було шо ми ирчі.', 'row': 4}),
 Document(page_content='hutsul_s

In [5]:
data_list = []
for i in range (len(data)):
    data_list.append(data[i].page_content.replace('hutsul_sentence: ', ''))
data_list[:20]

['У дєді з рєду дітий я двайціть и чєтвертий.',
 'А мижи живими дітми одинайцітий и найменчий, але ни випорток, бо післідь мене було ишє шестеро діточьок, лиш уни маленькими вивмерали.',
 'Ріс я пестуном у хаті, прото найменчя дитина у старині.',
 'Розвезений, шо най Бог боронит.',
 'Вара було шо ми ирчі.',
 'Бувало, скоро на мнє лиш шош трошки посиров’єт, то ого — я з хати порс тай просто до діда, маминого дєді.',
 'Добре, шо дорогу знав до діда, то ж страху ми ни було.',
 'Нираз малим хлопцем тікав до діда й без портєниц, скоро лиш ми та бига прийдет.',
 'Ніхто би був мнє ни спер.',
 'Дєдикові хати, статочьні збудованя, були побудовані у занодзі мижи ґрунями й лісами над бистро шумнючьов річьков.',
 'За річьков, наопередь самих викон, пид лісом тетеріла стримка, стряпава скала.',
 'Вид тої скали, поплямленої сивавими лішірями та порослої паршивими краками, покритої місцями зеленковатим мохом, було мині вічьно лєчьно.',
 'Найгірше було мині лєчьно вид печіри у тій скалі, бо та печіра 

In [6]:
from openai import OpenAI

client = OpenAI(
    api_key="your_key"
)

In [7]:
import os
os.environ["OPENAI_API_KEY"] = "your_key"

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Comment cell below if you do not need to perform embedding

In [ ]:
embedded_data = [embeddings.embed_query(text) for text in data_list]

In [ ]:
len(embedded_data[0])

1536

Save created embeddings (comment if not needed)

In [ ]:
embedded_numpy = np.array(embedded_data)
np.save('./embeddings/embeddings_openai.npy', embedded_numpy)

In [9]:
embedded_numpy = np.load('./embeddings/embeddings_openai.npy')
embedded_numpy

array([[ 0.03971342,  0.03673318, -0.02885518, ..., -0.03104993,
         0.00650917, -0.00817833],
       [ 0.01915574,  0.01543101, -0.07549359, ...,  0.00508213,
         0.02215289,  0.00896975],
       [ 0.02821364,  0.01712233, -0.02930831, ..., -0.01984869,
         0.02707766, -0.00199958],
       ...,
       [ 0.02536042,  0.07436955,  0.0213514 , ...,  0.00421172,
         0.02197077, -0.00674551],
       [ 0.03490249,  0.03160903, -0.00533376, ..., -0.00773137,
        -0.00647301, -0.01607379],
       [ 0.00596981,  0.02272311, -0.06663483, ..., -0.01765718,
         0.01030528, -0.00517366]])

In [10]:
len(embedded_numpy[0])

1536

In [11]:
# load it into FAISS
db_embeddings = embedded_numpy.copy().astype(np.float32)
faiss_index = faiss.IndexIDMap(faiss.IndexFlatIP(len(embedded_numpy[0])))
faiss.normalize_L2(db_embeddings)
faiss_index.add_with_ids(db_embeddings, np.arange(len(data_list)))

In [12]:
type(db_embeddings)

numpy.ndarray

In [13]:
len(db_embeddings[0])

1536

In [199]:
def index_search(query, k=4, threshold=0.45):
    query_vector = [query]
    search_query = np.array(query_vector).copy().astype(np.float32)
    faiss.normalize_L2(search_query)
    top_k = faiss_index.search(search_query, k)

    # Filter out sentences with similarity score lower than threshold
    filtered_results = [
        (sim_score, data_list[_id]) for sim_score, _id in zip(top_k[0][0], top_k[1][0]) if sim_score >= threshold
    ]

    # Reconstruct top_k with filtered results
    filtered_scores = [score for score, _ in filtered_results]
    filtered_sentences = [sentence for _, sentence in filtered_results]

    return (np.array([filtered_scores]), filtered_sentences[:min(len(filtered_sentences), k)])

In [15]:
print(faiss_index.index.ntotal)

11365


In [16]:
faiss_index

<faiss.swigfaiss.IndexIDMap; proxy of <Swig Object of type 'faiss::IndexIDMapTemplate< faiss::Index > *' at 0x137278f90> >

In [17]:
def csv_to_dictionary(csv_file):
    ukrainian_to_hutsul = {}
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            ukrainian = row[1].strip()
            hutsul = row[0].strip()
            ukrainian_to_hutsul[ukrainian] = hutsul
    return ukrainian_to_hutsul

In [18]:
csv_file = 'hutsul-ukrainian-dictionary.csv'
ukrainian_to_hutsul = csv_to_dictionary(csv_file)
print(ukrainian_to_hutsul)

{'Овва!': 'Ийга', 'Маланки': 'Їроди', 'Отвір': 'Кагла', 'Проповідь': 'Казання', 'Череватий': 'Калюхатий', 'Бутерброд': 'Канапка', 'Вид Зачіски': 'Канатка', 'Вуздечка Для Коня': 'Канейстра', 'Гармата': 'Канона', 'Кут Стола': 'Кант', 'Грязнуля': 'Капарник', 'Пластмаса': 'Карук', 'Скринька': 'Касетка', 'Метавка': 'Катапулька', 'Смертне Ложе': 'Катафейка', 'Чверть Літра': 'Кватирка', 'Мізки': 'Кебета', 'Гуцульський Одяг': 'Китиця', 'Гори': 'Кичери', 'Трогати': 'Ківати', 'Крокви': 'Кізли', 'Стегна': 'Клуби', 'Кусок Дерева': 'Кльоц', 'Клуб': 'Клюб', 'Ручка Дверей': 'Клямка', 'Клас': 'Кляса', 'Кнедлики': 'Книглі', 'Коли Б, Якщо Б': 'Кобих', 'Порожнина': 'Коварня', 'Залізничний': 'Колєйовий', 'Коляда': 'Коліда', 'Пастуша Колиба': 'Комарник', 'Поминки': 'Комашня', 'Димар': 'Комин', 'Католицька Церква': 'Костьол', 'Щиколотки': 'Котики', 'Вовняна Ковдра': 'Коц', 'Укривало': 'Коц', 'Колесо': 'Кочело', 'Кут Біля Печі': 'Кочерги', 'Качан': 'Кочєн', 'Пологи В Овець І Кіз': 'Кочіння', 'Загорожа Для Ві

In [19]:
def translate_to_hutsul(sentence, dictionary=ukrainian_to_hutsul):
    # Convert the dictionary keys to lowercase for case-insensitive matching
    dictionary_lower = {key.lower(): value for key, value in dictionary.items()}

    def replace_word(match):
        word = match.group(0)
        word_lower = word.lower()
        if word_lower in dictionary_lower:
            if word.islower():
                return dictionary_lower[word_lower].lower()
            elif word.isupper():
                return dictionary_lower[word_lower].upper()
            elif word.istitle():
                return dictionary_lower[word_lower].capitalize()
            else:
                return dictionary_lower[word_lower]
        return word

    translated_sentence = re.sub(r'\w+\'?\w*', replace_word, sentence)
    return translated_sentence

In [21]:
sentences_df = pd.read_csv("./rag-data/finction-10k.csv")
sentences_df.head(10)

,Sentence,Count,Found Phrases,NumWords,ChangedWords,ChangedWordsPercentage
0,Ось ми його зараз…,4,ось|зараз|ми|його,4,4,100.0
1,"Але це, звичайно, неможливо",4,це|звичайно|неможливо|але,4,4,100.0
2,Чи що він думав?,4,він|чи|що|думав,4,4,100.0
3,Чи людина щось може,4,щось|чи|може|людина,4,4,100.0
4,тут тільки трохи ще..,4,тут|трохи|тільки|ще,4,4,100.0
5,Але я відаю її,4,відаю|її|я|але,4,4,100.0
6,Людина нічого не може,4,не може|нічого|може|людина,4,4,100.0
7,Це зовсім добрий бінокль,4,добрий|бінокль|зовсім|це,4,4,100.0
8,Все це так ясно..,4,все|ясно|це|так,4,4,100.0
9,"Цікаво, як це сталося",4,цікаво|як|це|сталося,4,4,100.0


In [22]:
sentences_df["Processed_Sentence"] = sentences_df["Sentence"].apply(translate_to_hutsul)
sentences_df.head(10)

,Sentence,Count,Found Phrases,NumWords,ChangedWords,ChangedWordsPercentage,Processed_Sentence
0,Ось ми його зараз…,4,ось|зараз|ми|його,4,4,100.0,Леда ми го зараз…
1,"Але це, звичайно, неможливо",4,це|звичайно|неможливо|але,4,4,100.0,"Айбо цисе, ашош, гляба"
2,Чи що він думав?,4,він|чи|що|думав,4,4,100.0,Ци шо вин гадав сми?
3,Чи людина щось може,4,щось|чи|може|людина,4,4,100.0,Ци людьина шош годен
4,тут тільки трохи ще..,4,тут|трохи|тільки|ще,4,4,100.0,гир іно маленько шє..
5,Але я відаю її,4,відаю|її|я|але,4,4,100.0,Айбо йа свідім ї
6,Людина нічого не може,4,не може|нічого|може|людина,4,4,100.0,Людьина нич не годен
7,Це зовсім добрий бінокль,4,добрий|бінокль|зовсім|це,4,4,100.0,Цисе прем файний окян
8,Все це так ясно..,4,все|ясно|це|так,4,4,100.0,Всьо цисе йо ага..
9,"Цікаво, як це сталося",4,цікаво|як|це|сталося,4,4,100.0,"Вере, єк цисе трафилося"


In [23]:
hutsul_synthetic_base = sentences_df[["Sentence", "Processed_Sentence"]].copy()
hutsul_synthetic_base.head()

,Sentence,Processed_Sentence
0,Ось ми його зараз…,Леда ми го зараз…
1,"Але це, звичайно, неможливо","Айбо цисе, ашош, гляба"
2,Чи що він думав?,Ци шо вин гадав сми?
3,Чи людина щось може,Ци людьина шош годен
4,тут тільки трохи ще..,гир іно маленько шє..


In [ ]:
def generate_prompt(processed_sentence, context_sentences):
    context_str = "\n".join(context_sentences)
    prompt = f"Write the following sentence:\n{processed_sentence}\n\nin the same language style as the following sentences:\n{context_str}\n Return ONLY rewritten sentence, no other comments"
    return prompt

In [218]:
def generate_ukrainian_prompt(sentence, processed_sentence, context_sentences):
    context_str = "\n - ".join(context_sentences)
    prompt = f"""Речення літературною українською: "{sentence}".
Після зміни деяких слів на гуцульські відповідники за допомогою словника, оновлене речення: 
"{processed_sentence}".
    
Перепиши речення: "{processed_sentence}" на гуцульський діалект, використовуючи основні особливості і правила гуцульського діалекту. Приклад подібних речень у гуцульському діалекті:
 - {context_str}

Не модифікуй слова, які були змінені за допомогою словника, працюй з іншими словами у реченні. У відповіді поверни ТІЛЬКИ переписане тобою речення у гуцульському діалекті, без жодних коментарів чи пояснень. Поверни ТІЛЬКИ переписане речення. 
"""

    return prompt

In [214]:
def generate_instructions():
    prompt = f"""Ти експерт у гуцульському діалекті, і твоя задача: переписувати надані речення гуцульським діалектом.

Ти застосовуєш такі основні особливості і правила гуцульського діалекту при переписуванні речень гуцульським діалектом:
- Заміна "є" замість "а", "я" (коли наголошене). Приклад: гуцульський - "єк", українською - "як"; гуцульський - "єгода", українською - "ягода"; гуцульський - "єйце", українською - "яйце"; гуцульський - "єрмарок", українською - "ярмарок".
- Заміна "е" або рідше "і" замість "и" (коли наголошене). Приклад: гуцульський - "абес", українською - "абись"; гуцульський - "жето", українською - "жито". 
- Заміна "і" на "и". Приклад: гуцульський - "Иван", українською - "Іван"; гуцульський - "иду", українською - "іду".
- Заміна "я" на "и" або "і": Приклад: гуцульський - "забаривсми-си", українською - "забарився-ся"; гуцульський - "най-си", українською - "най-ся"
- Заміна "ю" на "у". Приклад: гуцульський - "суда", українською - "сюди"; гуцульський - "вівцу", українською - "вівцю".
- "ві-" замість "ви-" у префіксі. Приклад: гуцульський - "вітратити", українською - "витратити".
- Заміна "д" на "ґ". Приклад: гуцульський - "ґівка", українською - "дівка"; гуцульський - "ґедьо", українською - "дідо".
- Заміна "к" на "ть". Приклад: гуцульський - "доньтя", українською - "донька".
- Пом'якшення шиплячих. Приклад: гуцульський - "чьо", українською - "чого"; гуцульський - "чєлєдь", українською - "челядь".
- "д" замість "ґ". Приклад: гуцульський - "ледінь", українською - "леґінь, легінь".
- "к" замість "т" перед "є", "і", іноді в кінці слова. Приклад: гуцульський - "крескєни", українською - "християни"; гуцульський - "кокє", українською - "котя"; гуцульський - "кєшко", українською - "тяжко".
- "ц" замість "ч" і навпаки. Приклад: гуцульський - "ци", українською - "чи"; гуцульський - "чибук", українською - "цибух".
- тверді "с", "ц", "з" замість пом'якшених. Приклад: гуцульський - "шос", українською - "шось"; гуцульський - "дес", українською - "десь"; гуцульський - "хлопец", українською - "хлопець".
- Заміна "и" на "і", і навпаки в множині іменників. Приклад: гуцульський - "ангелі", українською - "ангели"; гуцульський - "Жебівци", українською - "Жебівці".
- "нн", "н" замість "дн", "тн", "лн". Приклад: гуцульський - "перенний", українською - "передний"; гуцульський - "мeнник" або "мeнник", українською - "мелник".
- Іменники основ "-а", "-я" в орудному відмінку мають закінчення "-ов", "-ев": руков «рукою», землев «землею»;

Приклади твого перекладу з літературної української мови на гуцульський діалект:
Літературна українська: За річкою, саме наперед вікон, під лісом тетеріла стрімка стряпава скеля.
Гуцульський діалект: За річьков, наопередь самих викон, пид лісом тетеріла стримка, стряпава скала.

У своїй відповіді ти повертаєш ТІЛЬКИ переписане тобою речення у гуцульському діалекті, без коментарів та різних згадувань. Лише речення написане гуцульським діалектом.
"""

    return prompt

In [210]:
print(generate_instructions())

Ти експерт у гуцульському діалекті, і твоя задача: переписувати надані речення гуцульським діалектом.

Ти застосовуєш такі основні особливості і правила гуцульського діалекту при переписуванні речень гуцульським діалектом:
- є замість а, я (коли наголошене): єк "як", єгода "ягода", єйце "яйце", єрмарок "ярмарок"
- е або рідше і замість и (коли наголошене): абес "абись", жето "жито" 
- Заміна і на и: Иван "Іван", иду "іду"
- Заміна я на и або і: забаривсми-си, най-си
- Заміна ю на у: суда "сюди", вівцу "вівцю"
- "ві-" замість "ви-" у префіксі: вітратити "втратити"
- Заміна д на ґ: ґівка "дівка", ґедьо "дідо"
- Заміна к на ть: доньтя "донька"
- Пом'якшення шиплячих: чьо "чого", чєлєдь "челядь"
- д замість ґ: ледінь "леґінь, легінь"
- к замість т перед є, і, іноді в кінці слова: крескєни "християни", кокє "котя", кєшко "тяжко"
- ц замість ч і навпаки: ци «чи», чибук «цибух»
- тверді с, ц, з замість пом'якшених: шос «шось», дес «десь», хлопец «хлопець»
- Заміна и на і і навпаки в множині і

In [221]:
def chatGPT_api(messages):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=0.0
    )

    return chat_completion.choices[0].message

def send_llm_request(sentence, processed_sentence, context_sentences, instructions):
    content = generate_ukrainian_prompt(sentence, processed_sentence, context_sentences)
    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": content}
    ]
    synthetic_sentence = chatGPT_api(messages).content

    return synthetic_sentence

In [198]:
def search_hutsul_sentences(sentence, threshold=0.4, top_n_neighbours=4):
  docs = index_search(sentence, k=top_n_neighbours, threshold=threshold)
  return docs

In [179]:
sentence = sentences_df["Sentence"][9999]
sentence

'Вона відповіла, посміхаючись: — Яка несподівана зустріч'

In [180]:
query = sentences_df["Processed_Sentence"][9999]
query

'Уна відповіла, посміхаючись: — Ика несподівана стріча'

In [143]:
index_search(query, k=4)

(array([[0.5912436, 0.559446 , 0.5419863, 0.5353999]], dtype=float32), ['Прото він привиташка був, тай тимунь йиго всі любили.', 'Знав, єк си де обійти й зайти, тимунь йиго усі люде за березованє фалили й любили, та навиділи йиго у колєдниках.', 'Шо я був тому винен, шо уна вже була стара тай тимунь їй си ни хотіло так пидбрикувати, єк мині молодому?.', 'А єк приніс їй доброго бука, уна устала: «Я твоя люба».'])

In [196]:
context_sentences = search_hutsul_sentences(query)
context_sentences

(array([], shape=(1, 0), dtype=float64), [])

In [197]:
len(context_sentences[0][0])

0

In [93]:
scores_csv = ','.join(map(str, context_sentences[0].flatten()))
scores_csv

'0.40938908,0.40802425,0.40578496,0.3948965'

In [182]:
instruction = generate_instructions()
synthetic_sentence = send_llm_request(sentence, query, context_sentences[1], instruction)

Речення літературною українською: "Вона відповіла, посміхаючись: — Яка несподівана зустріч".
Після зміни деяких слів на гуцульські відповідники за допомогою словника, оновлене речення: 
"Уна відповіла, посміхаючись: — Ика несподівана стріча".
    
Перепиши речення: "Уна відповіла, посміхаючись: — Ика несподівана стріча".
на гуцульський діалект, використовуючи основні особливості і правила гуцульського діалекту. Приклад подібний речень у гуцульському діалекті:
 - Довгий, фіґляво всміхаючіси, витавси: «Єк дновав, Ива’?».
 - «Та ні!» — здивовано відповів Иванчік.
 - «Ни бреши! Иків би ти ни знав?» — єк ґайда, злісно гаркнув дідо на мене.
 - Вна, квап’єчіси, борзо йиму сказала: «Иванчіку — чуєш?.

У відповіді поверни ТІЛЬКИ переписане тобою речення у гуцульському діалекті, без жодних інших коментарів.



In [183]:
synthetic_sentence

'Уна відповіла, посміхаючись: — Ика несподівана стріча.'

In [189]:
processed_sentences = sentences_df["Processed_Sentence"].to_numpy()
len(processed_sentences)

10055

In [191]:
from tqdm import tqdm

embedded_processed_sentences = []
for text in tqdm(processed_sentences, desc="Embedding Progress"):
    embedded_processed_sentences.append(embeddings.embed_query(text))

Embedding Progress: 100%|██████████| 10055/10055 [46:34<00:00,  3.60it/s] 


In [192]:
embedded_numpy_processed = np.array(embedded_processed_sentences)
np.save('./embeddings_processed_openai_fiction.npy', embedded_numpy_processed)

In [202]:
sentences_df.head()

,Sentence,Count,Found Phrases,NumWords,ChangedWords,ChangedWordsPercentage,Processed_Sentence
0,Ось ми його зараз…,4,ось|зараз|ми|його,4,4,100.0,Леда ми го зараз…
1,"Але це, звичайно, неможливо",4,це|звичайно|неможливо|але,4,4,100.0,"Айбо цисе, ашош, гляба"
2,Чи що він думав?,4,він|чи|що|думав,4,4,100.0,Ци шо вин гадав сми?
3,Чи людина щось може,4,щось|чи|може|людина,4,4,100.0,Ци людьина шош годен
4,тут тільки трохи ще..,4,тут|трохи|тільки|ще,4,4,100.0,гир іно маленько шє..


In [ ]:
instruction = generate_instructions()
synthetic_sentences = []
top_matches_scores = []

for index, row in tqdm(sentences_df.iterrows(), total=len(sentences_df), desc="Processing"):
    sentence = row["Sentence"]
    processed = row["Processed_Sentence"]
    embedded_query = embedded_processed_sentences[index]

    context_sentences = search_hutsul_sentences(embedded_query)
    if (context_sentences[0] == 0).any():
        continue
 
    synthetic_sentence = send_llm_request(sentence, processed, context_sentences[1], instruction)
    # synthetic_sentences.append(synthetic_sentence)
    if set(synthetic_sentence.split()) == set(processed.split()):
        synthetic_sentences.append(processed)
    else:
        synthetic_sentences.append(synthetic_sentence)

    scores_csv = ','.join(map(str, context_sentences[0].flatten()))
    top_matches_scores.append(scores_csv)

hutsul_synthetic_base["Synthetic_Sentence"] = synthetic_sentences
hutsul_synthetic_base["Top_matches_scores"] = top_matches_scores
hutsul_synthetic_base.head()

In [ ]:
# Save the DataFrame to a CSV file
hutsul_synthetic_base.to_csv("hutsul_synthetic_base.csv", index=False)